In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings

warnings.filterwarnings("ignore")

# deepOF model evaluation

Given a dataset and a trained model, this notebook allows the user to 

* Load and inspect the different models (encoder, decoder, grouper, gmvaep)
* Visualize reconstruction quality for a given model
* Visualize a static latent space
* Visualize trajectories on the latent space for a given video
* sample from the latent space distributions and generate video clips showcasing generated data

In [3]:
import os

os.chdir(os.path.dirname("../"))

In [4]:
import deepof.data
import deepof.utils
import numpy as np
import pandas as pd
import tensorflow as tf
from collections import Counter
from sklearn.preprocessing import StandardScaler

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import umap

from ipywidgets import interactive, interact, HBox, Layout, VBox
from IPython import display
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
import seaborn as sns

from ipywidgets import interact

### 1. Define and run project

In [5]:
path = os.path.join("..", "..", "Desktop", "deepoftesttemp")
trained_network = os.path.join("..", "..", "Desktop")
exclude_bodyparts = ["Tail_1", "Tail_2", "Tail_tip", "Tail_base"]
window_size = 11

In [6]:
%%time
proj = deepof.data.project(
    path=path, smooth_alpha=0.99, exclude_bodyparts=exclude_bodyparts, arena_dims=[380],
)

CPU times: user 293 ms, sys: 24.4 ms, total: 317 ms
Wall time: 269 ms


In [7]:
%%time
proj = proj.run(verbose=True)
print(proj)

Loading trajectories...
Smoothing trajectories...
Interpolating outliers...
Iterative imputation of ocluded bodyparts...
Computing distances...
Computing angles...
Done!
deepof analysis of 2 videos
CPU times: user 2.75 s, sys: 118 ms, total: 2.87 s
Wall time: 757 ms


### 2. Load pretrained deepof model

In [8]:
# Set model parameters
encoding = 6
loss = "ELBO"
k = 25
pheno = 0
predictor = 0

In [9]:
[i for i in os.listdir(trained_network) if i.endswith("h5")]

['GMVAE_loss=ELBO_encoding=6_k=25_latreg=none_20210312-084005_final_weights.h5',
 'GMVAE_loss=ELBO_encoding=6_k=25_latreg=variance_20210312-090508_final_weights.h5',
 'GMVAE_loss=ELBO_encoding=6_k=25_latreg=categorical+variance_20210312-085926_final_weights.h5',
 'GMVAE_loss=ELBO_encoding=6_k=25_latreg=categorical_20210312-093339_final_weights.h5']

In [10]:
coords = proj.get_coords(center="Center", align="Spine_1", align_inplace=True)
coords = coords.preprocess(test_videos=0, window_step=1, window_size=11, shuffle=True)[
    0
]

In [11]:
encoder, decoder, grouper, gmvaep = deepof.models.SEQ_2_SEQ_GMVAE(
    loss=loss,
    number_of_components=k,
    compile_model=True,
    encoding=encoding,
    predictor=predictor,
    phenotype_prediction=pheno,
).build(coords.shape)[:4]

gmvaep.load_weights(
    os.path.join(
        trained_network, [i for i in os.listdir(trained_network) if i.endswith("h5")][0]
    )
)

In [12]:
# Uncomment to see model summaries
# encoder.summary()
# decoder.summary()
# grouper.summary()
# gmvaep.summary()

In [13]:
# Uncomment to plot model structure
def plot_model(model, name):
    tf.keras.utils.plot_model(
        model,
        to_file=os.path.join(
            path,
            "deepof_{}_{}.png".format(name, datetime.now().strftime("%Y%m%d-%H%M%S")),
        ),
        show_shapes=True,
        show_dtype=False,
        show_layer_names=True,
        rankdir="TB",
        expand_nested=True,
        dpi=200,
    )


# plot_model(encoder, "encoder")
# plot_model(decoder, "decoder")
# plot_model(grouper, "grouper")
# plot_model(gmvaep, "gmvaep")

### 4. Evaluate reconstruction (to be incorporated into deepof.evaluate)

In [14]:
# Auxiliary animation functions


def plot_mouse_graph(instant_x, instant_y, instant_rec_x, instant_rec_y, ax, edges):
    """Generates a graph plot of the mouse"""
    plots = []
    rec_plots = []
    for edge in edges:
        (temp_plot,) = ax.plot(
            [float(instant_x[edge[0]]), float(instant_x[edge[1]])],
            [float(instant_y[edge[0]]), float(instant_y[edge[1]])],
            color="#006699",
            linewidth=2.0,
        )
        (temp_rec_plot,) = ax.plot(
            [float(instant_rec_x[edge[0]]), float(instant_rec_x[edge[1]])],
            [float(instant_rec_y[edge[0]]), float(instant_rec_y[edge[1]])],
            color="red",
            linewidth=2.0,
        )
        plots.append(temp_plot)
        rec_plots.append(temp_rec_plot)
    return plots, rec_plots


def update_mouse_graph(x, y, rec_x, rec_y, plots, rec_plots, edges):
    """Updates the graph plot to enable animation"""

    for plot, edge in zip(plots, edges):
        plot.set_data(
            [float(x[edge[0]]), float(x[edge[1]])],
            [float(y[edge[0]]), float(y[edge[1]])],
        )
    for plot, edge in zip(rec_plots, edges):
        plot.set_data(
            [float(rec_x[edge[0]]), float(rec_x[edge[1]])],
            [float(rec_y[edge[0]]), float(rec_y[edge[1]])],
        )

In [15]:
# Display a video with the original data superimposed with the reconstructions

coords = proj.get_coords(center="Center", align="Spine_1", align_inplace=True)
random_exp = np.random.choice(list(coords.keys()), 1)[0]
print(random_exp)


def animate_mice_across_time(random_exp):

    # Define canvas
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))

    # Retrieve body graph
    edges = deepof.utils.connect_mouse_topview()

    for bpart in exclude_bodyparts:
        edges.remove_node(bpart)

    for limb in ["Left_fhip", "Right_fhip", "Left_bhip", "Right_bhip"]:
        edges.remove_edge("Center", limb)

    edges = edges.edges()

    # Compute observed and predicted data to plot
    data = coords[random_exp]
    coords_rec = coords.filter_videos([random_exp])
    data_prep = coords_rec.preprocess(
        test_videos=0, window_step=1, window_size=window_size, shuffle=False
    )[0]

    data_rec = gmvaep.predict(data_prep)
    data_rec = pd.DataFrame(coords_rec._scaler.inverse_transform(data_rec[:, 6, :]))
    data_rec.columns = data.columns
    data = pd.DataFrame(coords_rec._scaler.inverse_transform(data_prep[:, 6, :]))
    data.columns = data_rec.columns

    # Add Central coordinate, lost during alignment
    data["Center", "x"] = 0
    data["Center", "y"] = 0
    data_rec["Center", "x"] = 0
    data_rec["Center", "y"] = 0

    # Plot!
    init_x = data.xs("x", level=1, axis=1, drop_level=False).iloc[0, :]
    init_y = data.xs("y", level=1, axis=1, drop_level=False).iloc[0, :]
    init_rec_x = data_rec.xs("x", level=1, axis=1, drop_level=False).iloc[0, :]
    init_rec_y = data_rec.xs("y", level=1, axis=1, drop_level=False).iloc[0, :]

    plots, rec_plots = plot_mouse_graph(
        init_x, init_y, init_rec_x, init_rec_y, ax, edges
    )
    scatter = ax.scatter(
        x=np.array(init_x), y=np.array(init_y), color="#006699", label="Original"
    )
    rec_scatter = ax.scatter(
        x=np.array(init_rec_x),
        y=np.array(init_rec_y),
        color="red",
        label="Reconstruction",
    )

    # Update data in main plot
    def animation_frame(i):
        # Update scatter plot
        x = data.xs("x", level=1, axis=1, drop_level=False).iloc[i, :]
        y = data.xs("y", level=1, axis=1, drop_level=False).iloc[i, :]
        rec_x = data_rec.xs("x", level=1, axis=1, drop_level=False).iloc[i, :]
        rec_y = data_rec.xs("y", level=1, axis=1, drop_level=False).iloc[i, :]

        scatter.set_offsets(np.c_[np.array(x), np.array(y)])
        rec_scatter.set_offsets(np.c_[np.array(rec_x), np.array(rec_y)])
        update_mouse_graph(x, y, rec_x, rec_y, plots, rec_plots, edges)

        return scatter

    animation = FuncAnimation(fig, func=animation_frame, frames=250, interval=75,)

    ax.set_title("Original versus reconstructed data")
    ax.set_ylim(-100, 60)
    ax.set_xlim(-60, 60)
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    plt.legend()

    video = animation.to_html5_video()
    html = display.HTML(video)
    display.display(html)
    plt.close()


animate_mice_across_time(random_exp)

Test 1_s11


### 5. Evaluate latent space (to be incorporated into deepof.evaluate)

In [16]:
# Get encodings and groupings for the same random video as above
data_prep = coords.preprocess(
    test_videos=0, window_step=1, window_size=window_size, shuffle=False
)[0]

encodings = encoder.predict(data_prep)
groupings = grouper.predict(data_prep)
hard_groups = np.argmax(groupings, axis=1)

In [17]:
@interact(minimum_confidence=(0.0, 1.0, 0.01))
def plot_cluster_confidence(minimum_confidence):
    plt.figure(figsize=(12, 8))

    groups = hard_groups[np.max(groupings, axis=1) > minimum_confidence].flatten()
    groups = np.concatenate([groups, np.arange(25)])
    sns.countplot(groups)
    plt.xlabel("Cluster")
    plt.title("Training instances per cluster")
    plt.show()

interactive(children=(FloatSlider(value=0.5, description='minimum_confidence', max=1.0, step=0.01), Output()),…

The slider in the figure above lets you set the minimum confidence the model may yield when assigning a training instance to a cluster in order to be visualized.

In [18]:
# Plot real data in the latent space


@interact(
    samples=(1000, 10000, 500),
    minimum_confidence=(0.0, 0.99, 0.01),
    dim_red=["LDA", "PCA", "umap", "tSNE"],
)
def plot_cluster_confidence(samples, minimum_confidence, dim_red):
    if dim_red == "umap":
        reducer = umap.UMAP(n_components=2)
    elif dim_red == "LDA":
        reducer = LinearDiscriminantAnalysis(n_components=2)
    elif dim_red == "PCA":
        reducer = PCA(n_components=2)
    else:
        reducer = TSNE(n_components=2)

    encods = encodings[np.max(groupings, axis=1) > minimum_confidence]
    groups = groupings[np.max(groupings, axis=1) > minimum_confidence]
    hgroups = hard_groups[np.max(groupings, axis=1) > minimum_confidence].flatten()

    samples = np.random.choice(range(encods.shape[0]), samples)
    sample_enc = encods[samples, :]
    sample_grp = groups[samples, :]
    sample_hgr = hgroups[samples]

    if dim_red != "LDA":
        enc = reducer.fit_transform(sample_enc)
    else:
        enc = reducer.fit_transform(sample_enc, sample_hgr)

    plt.figure(figsize=(12, 8))

    sns.scatterplot(
        x=enc[:, 0],
        y=enc[:, 1],
        hue=sample_hgr,
        size=np.max(sample_grp, axis=1),
        sizes=(1, 100),
        palette="muted",
    )
    plt.xlabel("{} 1".format(dim_red))
    plt.ylabel("{} 2".format(dim_red))
    plt.suptitle("Static view of trained latent space")
    plt.show()

interactive(children=(IntSlider(value=5500, description='samples', max=10000, min=1000, step=500), FloatSlider…

In [19]:
def plot_mouse_graph(instant_x, instant_y, ax, edges):
    """Generates a graph plot of the mouse"""
    plots = []
    for edge in edges:
        (temp_plot,) = ax.plot(
            [float(instant_x[edge[0]]), float(instant_x[edge[1]])],
            [float(instant_y[edge[0]]), float(instant_y[edge[1]])],
            color="#006699",
            linewidth=2.0,
        )
        plots.append(temp_plot)
    return plots


def update_mouse_graph(x, y, plots, edges):
    """Updates the graph plot to enable animation"""

    for plot, edge in zip(plots, edges):
        plot.set_data(
            [float(x[edge[0]]), float(x[edge[1]])],
            [float(y[edge[0]]), float(y[edge[1]])],
        )

In [20]:
# Plot trajectory of a video in latent space


@interact(
    samples=(1000, 10000, 500),
    trajectory=(100, 500),
    trace=False,
    dim_red=["LDA", "PCA", "umap", "tSNE"],
)
def plot_cluster_confidence(samples, trajectory, trace, dim_red):
    if dim_red == "umap":
        reducer = umap.UMAP(n_components=2)
    elif dim_red == "LDA":
        reducer = LinearDiscriminantAnalysis(n_components=2)
    elif dim_red == "PCA":
        reducer = PCA(n_components=2)
    else:
        reducer = TSNE(n_components=2)

    sample_enc = encodings[:samples, :]
    sample_grp = groupings[:samples, :]
    sample_hgr = hard_groups[:samples]

    if dim_red != "LDA":
        enc = reducer.fit_transform(sample_enc)
    else:
        enc = reducer.fit_transform(sample_enc, sample_hgr)

    # Define two figures arranged horizontally
    fig, (ax, ax2) = plt.subplots(
        1, 2, figsize=(12, 8), gridspec_kw={"width_ratios": [3, 1.5]}
    )

    # Plot the animated embedding trajectory on the left
    sns.scatterplot(
        x=enc[:, 0],
        y=enc[:, 1],
        hue=sample_hgr,
        size=np.max(sample_grp, axis=1),
        sizes=(1, 100),
        palette="muted",
        ax=ax,
    )

    traj_init = enc[0, :]
    scatter = ax.scatter(
        x=[traj_init[0]], y=[traj_init[1]], s=100, color="red", edgecolor="black"
    )
    (lineplt,) = ax.plot([traj_init[0]], [traj_init[1]], color="red", linewidth=2.0)
    tracking_line_x = []
    tracking_line_y = []

    # Plot the initial data (before feeding it to the encoder) on the right
    edges = deepof.utils.connect_mouse_topview()

    for bpart in exclude_bodyparts:
        if bpart:
            edges.remove_node(bpart)

    for limb in ["Left_fhip", "Right_fhip", "Left_bhip", "Right_bhip"]:
        edges.remove_edge("Center", limb)
        if ("Tail_base", limb) in list(edges.edges()):
            edges.remove_edge("Tail_base", limb)

    edges = edges.edges()

    inv_coords = coords._scaler.inverse_transform(data_prep)[:, window_size // 2, :]
    data = pd.DataFrame(inv_coords, columns=coords[random_exp].columns)

    data["Center", "x"] = 0
    data["Center", "y"] = 0

    init_x = data.xs("x", level=1, axis=1, drop_level=False).iloc[0, :]
    init_y = data.xs("y", level=1, axis=1, drop_level=False).iloc[0, :]

    plots = plot_mouse_graph(init_x, init_y, ax2, edges)
    track = ax2.scatter(x=np.array(init_x), y=np.array(init_y), color="#006699",)

    # Update data in both plots
    def animation_frame(i):
        # Update scatter plot
        offset = enc[i, :]

        prev_t = scatter.get_offsets()[0]

        if trace:
            tracking_line_x.append([prev_t[0], offset[0]])
            tracking_line_y.append([prev_t[1], offset[1]])
            lineplt.set_xdata(tracking_line_x)
            lineplt.set_ydata(tracking_line_y)

        scatter.set_offsets(np.c_[np.array(offset[0]), np.array(offset[1])])
        
        x = data.xs("x", level=1, axis=1, drop_level=False).iloc[i, :]
        y = data.xs("y", level=1, axis=1, drop_level=False).iloc[i, :]
        track.set_offsets(np.c_[np.array(x), np.array(y)])
        update_mouse_graph(x, y, plots, edges)

        return scatter

    animation = FuncAnimation(
        fig, func=animation_frame, frames=trajectory, interval=75,
    )

    ax.set_xlabel("{} 1".format(dim_red))
    ax.set_ylabel("{} 2".format(dim_red))

    ax2.set_xlabel("x")
    ax2.set_xlabel("y")
    ax2.set_ylim(-90, 60)
    ax2.set_xlim(-60, 60)

    plt.tight_layout()

    video = animation.to_html5_video()
    html = display.HTML(video)
    display.display(html)
    plt.close()

interactive(children=(IntSlider(value=5500, description='samples', max=10000, min=1000, step=500), IntSlider(v…

### 6. Sample from latent space (to be incorporated into deepof.evaluate)

In [ ]:
def retrieve_latent_parameters(
    distribution, reduce=False, plot=False, categories=None, filt=0, save=True
):
    mix_components = distribution.components

    # The main problem is here! We need to select only those training instances in which a given cluster was selected.
    # Then compute the median for those only

    means = [
        get_median_params(component, categories, i, "mean")
        for i, component in enumerate(mix_components)
    ]
    stddevs = [
        get_median_params(component, categories, i, "stddev")
        for i, component in enumerate(mix_components)
    ]
    means = [i for i in means if i is not None]
    stddevs = [i for i in stddevs if i is not None]

    if filter:
        filts = np.max(categories, axis=0) > filt
        means = [i for i, j in zip(means, filts) if j]
        stddevs = [i for i, j in zip(stddevs, filts) if j]

    if reduce:
        data = [
            np.random.normal(size=[1000, len(means[0])], loc=meanvec, scale=stddevvec)[
                :, np.newaxis
            ]
            for meanvec, stddevvec in zip(means, stddevs)
        ]
        data = np.concatenate(data, axis=1).reshape([1000 * len(means), len(means[0])])
        reducer = PCA(n_components=3)
        data = reducer.fit_transform(data)
        data = data.reshape([1000, len(means), 3])

    if plot == 2:
        for i in range(len(means)):
            plt.scatter(data[:, i, 0], data[:, i, 1], label=i)
        plt.title(
            "Mean representation of latent space - K={}/{} - L={} - filt={}".format(
                len(means), len(mix_components), len(means[0]), filt
            )
        )
        plt.xlabel("PCA 1")
        plt.ylabel("PCA 2")
        # plt.legend()
        if save:
            plt.savefig(
                "Mean representation of latent space - K={}.{} - L={} - filt={}.png".format(
                    len(means), len(mix_components), len(means[0]), filt
                ).replace(
                    " ", "_"
                )
            )
        plt.show()

    elif plot == 3:
        fig = plt.figure()
        ax = fig.add_subplot(111, projection="3d")
        for i in range(len(means)):
            ax.scatter(data[:, i, 0], data[:, i, 1], data[:, i, 2], label=i)
        plt.title(
            "Mean representation of latent space - K={}/{} - L={} - filt={}".format(
                len(means), len(mix_components), len(means[0]), filt
            )
        )
        ax.set_xlabel("PCA 1")
        ax.set_ylabel("PCA 2")
        ax.set_zlabel("PCA 3")
        # plt.legend()
        if save:
            plt.savefig(
                "Mean representation of latent space - K={}.{} - L={} - filt={}.png".format(
                    len(means), len(mix_components), len(means[0]), filt
                ).replace(
                    " ", "_"
                )
            )
        plt.show()

    elif plot > 3:
        raise ValueError("Can't plot in more than 3 dimensions!")

    return means, stddevs


def sample_from_posterior(
    decoder, parameters, component, enable_variance=False, video_output=False, samples=1
):
    means, stddevs = parameters
    sample = np.random.normal(
        size=[samples, len(means[component])],
        loc=means[component],
        scale=(stddevs[component] if enable_variance else 0),
    )
    reconstruction = decoder(sample).mean()

    if video_output:
        scaled_video_rec = scaler.inverse_transform(reconstruction)
        scaled_video_rec = scaled_video_rec.reshape(
            [samples * scaled_video_rec.shape[1], scaled_video_rec.shape[2]]
        )
        columns = deepof_coords[list(deepof_coords.keys())[0]].columns
        scaled_video_rec = pd.DataFrame(scaled_video_rec, columns=columns)

        ### VIDEO OUTPUT ###
        w = 400
        h = 400
        factor = 2.5

        # Instantiate video
        writer = cv2.VideoWriter()
        writer.open(
            "Reconstruction_test_L={}_k={}_pheno={}_component={}_video.avi".format(
                l, k, pheno, component
            ),
            cv2.VideoWriter_fourcc(*"MJPG"),
            24,
            (int(w * factor), int(h * factor)),
            True,
        )

        for frame in tqdm.tqdm(range(scaled_video_rec.shape[0])):

            image = np.zeros((h, w, 3), np.uint8) + 30
            for bpart in scaled_video_rec.columns.levels[0]:

                try:
                    pos = (
                        (-int(scaled_video_rec[bpart].loc[frame, "x"]) + w // 2),
                        (-int(scaled_video_rec[bpart].loc[frame, "y"]) + h // 2),
                    )

                    cv2.circle(image, pos, 2, (0, 0, 255), -1)

                except KeyError:
                    continue

            # draw skeleton
            def draw_line(start, end, df, col):
                for bpart in end:
                    cv2.line(
                        image,
                        tuple(-df[start].loc[frame, :].asisinstance(int) + w // 2),
                        tuple(-df[bpart].loc[frame, :].asisinstance(int) + h // 2),
                        col,
                        1,
                    )

            col = (0, 0, 255)
            draw_line("Nose", ["Left_ear", "Right_ear"], scaled_video_rec, col)
            draw_line(
                "Spine_1",
                ["Left_ear", "Right_ear", "Left_fhip", "Right_fhip"],
                scaled_video_rec,
                col,
            )
            draw_line(
                "Spine_2", ["Spine_1", "Left_bhip", "Right_bhip"], scaled_video_rec, col
            )
            # draw_line("Tail_1", ["Tail_base", "Tail_2"], scaled_video_rec, col)
            # draw_line("Tail_tip", ["Tail_2"], scaled_video_rec, col)

            image = cv2.resize(image, (0, 0), fx=factor, fy=factor)
            writer.write(image)

        writer.release()
        cv2.destroyAllWindows()

    return reconstruction

In [29]:
print(encoder.output)

NotImplementedError: 

In [25]:
trained_distribution = encoder.predict(data_prep)
#categories = tf.keras.models.Model(encoder.input, encoder.layers[15].output)(
#    data_prep
#).numpy()

array([[ 0.69863343, -0.24356522, -2.0744514 ,  4.3458786 ,  1.1873256 ,
        -0.8186187 ],
       [ 0.6072135 , -0.18216974, -2.1706762 ,  3.1971736 ,  1.0388865 ,
        -0.10271598],
       [ 0.603236  , -0.08107454, -2.2042763 ,  3.3883073 ,  1.0370435 ,
        -0.02851732],
       ...,
       [ 0.62569886, -0.5773468 ,  0.09486004,  7.658196  ,  1.3188459 ,
        -2.6761112 ],
       [ 0.5992341 , -1.0350724 ,  0.50989425,  7.7878056 ,  1.3219635 ,
        -2.5545087 ],
       [ 0.4544863 , -1.4881462 ,  0.9579054 ,  7.7931585 ,  1.302621  ,
        -2.4311593 ]], dtype=float32)

In [ ]:
means, stddevs = retrieve_latent_parameters(
    trained_distribution,
    categories=categories,
    reduce=True,
    plot=2,
    filt=0.9,
    save=True,
)